In [3]:
import pandas as pd

# Load accelerometer data
acc = pd.read_csv('sensor_data/Sitting_2-2026-01-12_11-14-27/AccelerometerUncalibrated.csv')
# Load gyroscope data
gyro = pd.read_csv('sensor_data/Sitting_2-2026-01-12_11-14-27/GyroscopeUncalibrated.csv')

# Normalize column names to lowercase with underscores
acc.columns = [c.strip().lower().replace(' ', '_') for c in acc.columns]
gyro.columns = [c.strip().lower().replace(' ', '_') for c in gyro.columns]

# Unify timestamp column name from 'time' to 'timestamp'
if 'time' in acc.columns:
    acc = acc.rename(columns={'time': 'timestamp'})
if 'time' in gyro.columns:
    gyro = gyro.rename(columns={'time': 'timestamp'})

# Merge on timestamp; suffixes keep accel/gyro axes separate
merged = pd.merge(acc, gyro, on='timestamp', suffixes=('_acc', '_gyro'))

# Debug: Print available columns after merge
print("Columns after merge:", merged.columns.tolist())

# Dynamically build rename map based on what actually exists
rename_map = {}
for col in merged.columns:
    if col.endswith('_acc'):
        base = col.replace('_acc', '')
        rename_map[col] = f'acc_{base}'
    elif col.endswith('_gyro'):
        base = col.replace('_gyro', '')
        rename_map[col] = f'gyro_{base}'

# Handle seconds_elapsed - it might be seconds_elapsed or seconds_elapsed_acc
if 'seconds_elapsed_acc' in merged.columns:
    rename_map['seconds_elapsed_acc'] = 'seconds_elapsed'
elif 'seconds_elapsed' not in rename_map:
    # seconds_elapsed already exists without suffix, no rename needed
    pass

print("Rename map:", rename_map)
merged = merged.rename(columns=rename_map)

print("Columns after rename:", merged.columns.tolist())

# Select and order the output columns
output_cols = [
    'timestamp',
    'seconds_elapsed',
    'acc_z',
    'acc_y',
    'acc_x',
    'gyro_z',
    'gyro_y',
    'gyro_x'
]

merged = merged[output_cols]

# Save to CSV (semicolon separated, like standing2_data.csv)
merged.to_csv('./sensor_data/sitting_data.csv', sep=';', index=False)

print("Merged data saved to 'merged_sensor_data.csv'")

Columns after merge: ['timestamp', 'seconds_elapsed_acc', 'z_acc', 'y_acc', 'x_acc', 'seconds_elapsed_gyro', 'z_gyro', 'y_gyro', 'x_gyro']
Rename map: {'seconds_elapsed_acc': 'seconds_elapsed', 'z_acc': 'acc_z', 'y_acc': 'acc_y', 'x_acc': 'acc_x', 'seconds_elapsed_gyro': 'gyro_seconds_elapsed', 'z_gyro': 'gyro_z', 'y_gyro': 'gyro_y', 'x_gyro': 'gyro_x'}
Columns after rename: ['timestamp', 'seconds_elapsed', 'acc_z', 'acc_y', 'acc_x', 'gyro_seconds_elapsed', 'gyro_z', 'gyro_y', 'gyro_x']
Merged data saved to 'merged_sensor_data.csv'
